## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [53]:
# Get today's date
today = datetime.now() - timedelta(days=2)

# Get the date one week ago
one_week_ago = today - timedelta(days=8)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

May 30 June 7


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-2-2025,Condo/Co-op,1300 Brickell Bay Dr #2404,Miami,FL,33131.0,640000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1300-Brickell-...,MARMLS,A11763087,N,Y,25.760372,-80.190197


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-06-02    31
2025-06-06    31
2025-06-05    23
2025-06-03    21
2025-06-04    17
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
77,PAST SALE,2025-06-06,Condo/Co-op,7852 Fisher Island Dr #7852,Miami Beach,FL,33109.0,12000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,MARMLS,A11735076,N,Y,25.759774,-80.137270
86,PAST SALE,2025-06-05,Condo/Co-op,18201 Collins Ave #409,Sunny Isles Beach,FL,33160.0,5600000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11692539,N,Y,25.944933,-80.120001
98,PAST SALE,2025-06-02,Condo/Co-op,1000 S Pointe Dr #501,Miami Beach,FL,33139.0,5270000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11692555,N,Y,25.768456,-80.137440
103,PAST SALE,2025-06-06,Condo/Co-op,100 S Pointe Dr #907,Miami Beach,FL,33139.0,4800000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11658722,N,Y,25.767180,-80.132822
122,PAST SALE,2025-06-02,Condo/Co-op,2900 NE 7th Ave #407,Miami,FL,33137.0,3500000.0,5.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11404730,N,Y,25.805016,-80.185942
115,PAST SALE,2025-06-03,Condo/Co-op,901 Brickell Key Blvd #3508,Miami,FL,33131.0,3340000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11629987,N,Y,25.769299,-80.186901
16,PAST SALE,2025-06-02,Condo/Co-op,19225 Fisher Island Dr #19225,Fisher Island,FL,33109.0,3300000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,MARMLS,A11789231,N,Y,25.757729,-80.138605
94,PAST SALE,2025-06-06,Condo/Co-op,2600 Island Blvd #2306,Aventura,FL,33160.0,3100000.0,5.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2600-Island...,MARMLS,A11677170,N,Y,25.939020,-80.133755
78,PAST SALE,2025-06-02,Condo/Co-op,1000 Brickell Plz Ph 5003,Miami,FL,33131.0,2736000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11707365,N,Y,25.764264,-80.192725
6,PAST SALE,2025-06-06,Condo/Co-op,6000 Island Blvd #2102,Aventura,FL,33160.0,2700000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11794732,N,Y,25.942761,-80.135433


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [18]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [19]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [20]:
response_df = pd.DataFrame(response_list)

In [21]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [22]:
df_top_ten = merged_df

## Current Week's Values

In [29]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

123


In [30]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$935,700


In [31]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$581


In [32]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$115,091,100


In [33]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [34]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
129
-----------
Input Previous Week Condo Average Sales Price:
1173360
-----------
Input Previous Week Condo Average PSF:
657
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
151363476.0


In [36]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,129,1173360,657,151363476.0


In [35]:
last_week['Last Week Total Dollar Volume'].iloc[0]

151363476.0

In [37]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          123              935700          581                    115091100.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [38]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [39]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [41]:
pd.set_option('display.max_columns',None)

In [42]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-06,Condo/Co-op,7852 Fisher Island Dr #7852,Miami Beach,FL,33109.0,12000000.0,3.0,3.5,FISHER ISLAND,4590.0,NaN,1990.0,NaN,2614.0,11710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,MARMLS,A11735076,N,Y,25.759774,-80.13727,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,Jill Eber,Coldwell Banker Realty,None,None,Alexa Iacovelli,Luxe Living Realty,None,None,1,orange


In [43]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-06,Condo/Co-op,7852 Fisher Island Dr #7852,Miami Beach,FL,33109.0,12000000.0,3.0,3.5,FISHER ISLAND,4590.0,NaN,1990.0,NaN,2614.0,11710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,MARMLS,A11735076,N,Y,25.759774,-80.137270,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,Jill Eber,Coldwell Banker Realty,None,None,Alexa Iacovelli,Luxe Living Realty,None,None,1,orange
1,PAST SALE,2025-06-05,Condo/Co-op,18201 Collins Ave #409,Sunny Isles Beach,FL,33160.0,5600000.0,4.0,5.5,TRUMP ROYALE CONDO,3087.0,NaN,2009.0,NaN,1814.0,3737.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11692539,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Daniel Tzinker,The Agency Florida LLC,None,None,2,blue
2,PAST SALE,2025-06-02,Condo/Co-op,1000 S Pointe Dr #501,Miami Beach,FL,33139.0,5270000.0,3.0,3.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,1566.0,9156.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11692555,N,Y,25.768456,-80.137440,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Jill Hertzberg,Coldwell Banker Realty,None,None,MAR NON-MEMBER,NO MLS PARTICIPATION (MAR),None,None,3,blue
3,PAST SALE,2025-06-06,Condo/Co-op,100 S Pointe Dr #907,Miami Beach,FL,33139.0,4800000.0,2.0,2.5,Continuum South,1870.0,NaN,2003.0,NaN,2567.0,4579.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11658722,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Sonia Toth,Brown Harris Stevens,None,None,Taylor Richardson,The Corcoran Group,None,None,4,blue
4,PAST SALE,2025-06-02,Condo/Co-op,2900 NE 7th Ave #407,Miami,FL,33137.0,3500000.0,5.0,5.5,BISCAYNE BEACH CONDO,4146.0,NaN,2017.0,NaN,844.0,6075.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11404730,N,Y,25.805016,-80.185942,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",5,blue
5,PAST SALE,2025-06-03,Condo/Co-op,901 Brickell Key Blvd #3508,Miami,FL,33131.0,3340000.0,4.0,4.5,CARBONELL CONDO,2860.0,NaN,2005.0,NaN,1168.0,2973.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11629987,N,Y,25.769299,-80.186901,https://www.redfin.com/FL/Miami/901-Brickell-K...,Joao Carvalho,"Jad Realty, LLC.",Daniela Carvalho,"Jad Realty, LLC.",Emerson De Lima,Cervera Real Estate Inc.,None,None,6,blue
6,PAST SALE,2025-06-02,Condo/Co-op,19225 Fisher Island Dr #19225,Fisher Island,FL,33109.0,3300000.0,2.0,2.0,SEASIDE CONDO,1875.0,NaN,1989.0,NaN,1760.0,7269.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,MARMLS,A11789231,N,Y,25.757729,-80.138605,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Giovanna Ingletto Amaral,RGI Realty,None,None,7,blue
7,PAST SALE,2025-06-06,Condo/Co-op,2600 Island Blvd #2306,Aventura,FL,33160.0,3100000.0,5.0,5.5,2600 ISL BLVD WILLIAMS IS,4740.0,NaN,1997.0,NaN,654.0,6243.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2600-Island...,MARMLS,A11677170,N,Y,25.939020,-80.133755,https://www.redfin.com/FL/Aventura/2600-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Caryn Botknecht,"Compass Florida, LLC",None,None,8,blue
8,PAST SALE,2025-06-02,Condo/Co-op,1000 Brickell Plz Ph 5003,Miami,FL,33131.0,2736000.0,3.0,4.5,BRICKELL FLATIRON CONDO,2076.0,NaN,2019.0,

In [58]:
%store -r map_box_api_key

In [65]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [66]:
m.save('index.html')

## Data snagger

In [67]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [68]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-06,Condo/Co-op,7852 Fisher Island Dr #7852,Miami Beach,FL,33109.0,12000000.0,3.0,3.5,FISHER ISLAND,4590.0,NaN,1990.0,NaN,2614.0,11710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,MARMLS,A11735076,N,Y,25.759774,-80.13727,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,Jill Eber,Coldwell Banker Realty,None,None,Alexa Iacovelli,Luxe Living Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [69]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_06092025 


# CREATE TEMPLATE 

In [70]:
muni_set = set(df_top_ten['CITY'])

In [71]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [72]:
df_top_ten.reset_index(inplace=True,drop=True)

In [73]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [74]:
top_sale

'FISHER ISLAND at 7852 Fisher Island Dr #7852 in Miami Beach'

In [75]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [76]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [77]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-06,Condo/Co-op,7852 Fisher Island Dr #7852,Miami Beach,FL,33109.0,12000000.0,3.0,3.5,FISHER ISLAND,4590.0,NaN,1990.0,NaN,2614.0,11710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,MARMLS,A11735076,N,Y,25.759774,-80.137270,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,Jill Eber,Coldwell Banker Realty,None,None,Alexa Iacovelli,Luxe Living Realty,None,None,1,orange,FISHER ISLAND at 7852 Fisher Island Dr #7852 i...
1,PAST SALE,2025-06-05,Condo/Co-op,18201 Collins Ave #409,Sunny Isles Beach,FL,33160.0,5600000.0,4.0,5.5,TRUMP ROYALE CONDO,3087.0,NaN,2009.0,NaN,1814.0,3737.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11692539,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Daniel Tzinker,The Agency Florida LLC,None,None,2,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #409 i...
2,PAST SALE,2025-06-02,Condo/Co-op,1000 S Pointe Dr #501,Miami Beach,FL,33139.0,5270000.0,3.0,3.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,1566.0,9156.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11692555,N,Y,25.768456,-80.137440,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Jill Hertzberg,Coldwell Banker Realty,None,None,MAR NON-MEMBER,NO MLS PARTICIPATION (MAR),None,None,3,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
3,PAST SALE,2025-06-06,Condo/Co-op,100 S Pointe Dr #907,Miami Beach,FL,33139.0,4800000.0,2.0,2.5,Continuum South,1870.0,NaN,2003.0,NaN,2567.0,4579.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11658722,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Sonia Toth,Brown Harris Stevens,None,None,Taylor Richardson,The Corcoran Group,None,None,4,blue,Continuum South at 100 S Pointe Dr #907 in Mia...
4,PAST SALE,2025-06-02,Condo/Co-op,2900 NE 7th Ave #407,Miami,FL,33137.0,3500000.0,5.0,5.5,BISCAYNE BEACH CONDO,4146.0,NaN,2017.0,NaN,844.0,6075.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11404730,N,Y,25.805016,-80.185942,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",5,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #407 i...
5,PAST SALE,2025-06-03,Condo/Co-op,901 Brickell Key Blvd #3508,Miami,FL,33131.0,3340000.0,4.0,4.5,CARBONELL CONDO,2860.0,NaN,2005.0,NaN,1168.0,2973.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11629987,N,Y,25.769299,-80.186901,https://www.redfin.com/FL/Miami/901-Brickell-K...,Joao Carvalho,"Jad Realty, LLC.",Daniela Carvalho,"Jad Realty, LLC.",Emerson De Lima,Cervera Real Estate Inc.,None,None,6,blue,CARBONELL CONDO at 901 Brickell Key Blvd #3508...
6,PAST SALE,2025-06-02,Condo/Co-op,19225 Fisher Island Dr #19225,Fisher Island,FL,33109.0,3300000.0,2.0,2.0,SEASIDE CONDO,1875.0,NaN,1989.0,NaN,1760.0,7269.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,MARMLS,A11789231,N,Y,25.757729,-80.138605,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Giovanna Ingletto Amaral,RGI Realty,None,None,7,blue,SEASIDE CONDO at 19225 Fisher Island Dr #19225...
7,PAST SALE,2025-06-06,Condo/Co-op,2600 Island Blvd #2306,Aventura,FL,33160.0,3100000.0,5.0,5.5,2600 ISL BLVD WILLIAMS IS,4740.0,NaN,1997.0,NaN,654.0,6243.0,Sold,NaN,NaN,https://www.redfin.com/FL/A

In [78]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: FISHER ISLAND closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,700,000 to $12,000,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Fisher Island, Miami Beach, Aventura, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 123 condo sales totaling $115,091,100 million from May 30 to June 7. The previous week, brokers closed 129 condo sales totaling $151,363,476.

Last week’s units sold for an average of $935,700, lower than the $1,173,360 average price from 

In [79]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [80]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [81]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [82]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-06,Condo/Co-op,7852 Fisher Island Dr #7852,Miami Beach,FL,33109.0,12000000.0,3.0,3.5,FISHER ISLAND,4590.0,NaN,1990.0,NaN,2614.0,11710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,MARMLS,A11735076,N,Y,25.759774,-80.137270,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,Jill Eber,Coldwell Banker Realty,None,None,Alexa Iacovelli,Luxe Living Realty,None,None,1,orange,FISHER ISLAND at 7852 Fisher Island Dr #7852 i...
1,PAST SALE,2025-06-05,Condo/Co-op,18201 Collins Ave #409,Sunny Isles Beach,FL,33160.0,5600000.0,4.0,5.5,TRUMP ROYALE CONDO,3087.0,NaN,2009.0,NaN,1814.0,3737.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11692539,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Daniel Tzinker,The Agency Florida LLC,None,None,2,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #409 i...
2,PAST SALE,2025-06-02,Condo/Co-op,1000 S Pointe Dr #501,Miami Beach,FL,33139.0,5270000.0,3.0,3.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,1566.0,9156.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11692555,N,Y,25.768456,-80.137440,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Jill Hertzberg,Coldwell Banker Realty,None,None,MAR NON-MEMBER,NO MLS PARTICIPATION (MAR),None,None,3,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
3,PAST SALE,2025-06-06,Condo/Co-op,100 S Pointe Dr #907,Miami Beach,FL,33139.0,4800000.0,2.0,2.5,Continuum South,1870.0,NaN,2003.0,NaN,2567.0,4579.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11658722,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Sonia Toth,Brown Harris Stevens,None,None,Taylor Richardson,The Corcoran Group,None,None,4,blue,Continuum South at 100 S Pointe Dr #907 in Mia...
4,PAST SALE,2025-06-02,Condo/Co-op,2900 NE 7th Ave #407,Miami,FL,33137.0,3500000.0,5.0,5.5,BISCAYNE BEACH CONDO,4146.0,NaN,2017.0,NaN,844.0,6075.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11404730,N,Y,25.805016,-80.185942,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",5,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #407 i...
5,PAST SALE,2025-06-03,Condo/Co-op,901 Brickell Key Blvd #3508,Miami,FL,33131.0,3340000.0,4.0,4.5,CARBONELL CONDO,2860.0,NaN,2005.0,NaN,1168.0,2973.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11629987,N,Y,25.769299,-80.186901,https://www.redfin.com/FL/Miami/901-Brickell-K...,Joao Carvalho,"Jad Realty, LLC.",Daniela Carvalho,"Jad Realty, LLC.",Emerson De Lima,Cervera Real Estate Inc.,None,None,6,blue,CARBONELL CONDO at 901 Brickell Key Blvd #3508...
6,PAST SALE,2025-06-02,Condo/Co-op,19225 Fisher Island Dr #19225,Fisher Island,FL,33109.0,3300000.0,2.0,2.0,SEASIDE CONDO,1875.0,NaN,1989.0,NaN,1760.0,7269.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,MARMLS,A11789231,N,Y,25.757729,-80.138605,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Giovanna Ingletto Amaral,RGI Realty,None,None,7,blue,SEASIDE CONDO at 19225 Fisher Island Dr #19225...
7,PAST SALE,2025-06-06,Condo/Co-op,2600 Island Blvd #2306,Aventura,FL,33160.0,3100000.0,5.0,5.5,2600 ISL BLVD WILLIAMS IS,4740.0,NaN,1997.0,NaN,654.0,6243.0,Sold,NaN,NaN,https://www.redfin.com/FL/A

In [83]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: FISHER ISLAND closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,700,000 to $12,000,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Fisher Island, Miami Beach, Aventura, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 123 condo sales totaling $115,091,100 million from May 30 to June 7. The previous week, brokers closed 129 condo sales totaling $151,363,476.

Last week’s units sold for an average of $935,700, lower than the $1,173,360 average price from 

In [84]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami-Beach/7852-Fisher-Island-Dr-33109/unit-7852/home/43096875


In [85]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/18201-Collins-Ave-33160/unit-409/home/42675793


In [87]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Aventura/6000-Island-Blvd-33160/unit-2102/home/43344586


In [88]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/1000-Brickell-Plz-33131/unit-5003/home/79135651


In [89]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/7852-Fisher-Island-Dr-33109/unit-7852/home/43096875


In [90]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/2600-Island-Blvd-33160/unit-2306/home/42942076


In [91]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-06,Condo/Co-op,7852 Fisher Island Dr #7852,Miami Beach,FL,33109.0,12000000.0,3.0,3.5,FISHER ISLAND,4590.0,NaN,1990.0,NaN,2614.0,11710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,MARMLS,A11735076,N,Y,25.759774,-80.137270,https://www.redfin.com/FL/Miami-Beach/7852-Fis...,Jill Eber,Coldwell Banker Realty,None,None,Alexa Iacovelli,Luxe Living Realty,None,None,1,orange,FISHER ISLAND at 7852 Fisher Island Dr #7852 i...
1,PAST SALE,2025-06-05,Condo/Co-op,18201 Collins Ave #409,Sunny Isles Beach,FL,33160.0,5600000.0,4.0,5.5,TRUMP ROYALE CONDO,3087.0,NaN,2009.0,NaN,1814.0,3737.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11692539,N,Y,25.944933,-80.120001,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Daniel Tzinker,The Agency Florida LLC,None,None,2,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #409 i...
2,PAST SALE,2025-06-02,Condo/Co-op,1000 S Pointe Dr #501,Miami Beach,FL,33139.0,5270000.0,3.0,3.5,MURANO AT PORTOFINO CONDO,3365.0,NaN,2002.0,NaN,1566.0,9156.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11692555,N,Y,25.768456,-80.137440,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,Jill Hertzberg,Coldwell Banker Realty,None,None,MAR NON-MEMBER,NO MLS PARTICIPATION (MAR),None,None,3,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
3,PAST SALE,2025-06-06,Condo/Co-op,100 S Pointe Dr #907,Miami Beach,FL,33139.0,4800000.0,2.0,2.5,Continuum South,1870.0,NaN,2003.0,NaN,2567.0,4579.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11658722,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Sonia Toth,Brown Harris Stevens,None,None,Taylor Richardson,The Corcoran Group,None,None,4,blue,Continuum South at 100 S Pointe Dr #907 in Mia...
4,PAST SALE,2025-06-02,Condo/Co-op,2900 NE 7th Ave #407,Miami,FL,33137.0,3500000.0,5.0,5.5,BISCAYNE BEACH CONDO,4146.0,NaN,2017.0,NaN,844.0,6075.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11404730,N,Y,25.805016,-80.185942,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",5,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #407 i...
5,PAST SALE,2025-06-03,Condo/Co-op,901 Brickell Key Blvd #3508,Miami,FL,33131.0,3340000.0,4.0,4.5,CARBONELL CONDO,2860.0,NaN,2005.0,NaN,1168.0,2973.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11629987,N,Y,25.769299,-80.186901,https://www.redfin.com/FL/Miami/901-Brickell-K...,Joao Carvalho,"Jad Realty, LLC.",Daniela Carvalho,"Jad Realty, LLC.",Emerson De Lima,Cervera Real Estate Inc.,None,None,6,blue,CARBONELL CONDO at 901 Brickell Key Blvd #3508...
6,PAST SALE,2025-06-02,Condo/Co-op,19225 Fisher Island Dr #19225,Fisher Island,FL,33109.0,3300000.0,2.0,2.0,SEASIDE CONDO,1875.0,NaN,1989.0,NaN,1760.0,7269.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,MARMLS,A11789231,N,Y,25.757729,-80.138605,https://www.redfin.com/FL/Miami-Beach/19225-Fi...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Giovanna Ingletto Amaral,RGI Realty,None,None,7,blue,SEASIDE CONDO at 19225 Fisher Island Dr #19225...
7,PAST SALE,2025-06-06,Condo/Co-op,2600 Island Blvd #2306,Aventura,FL,33160.0,3100000.0,5.0,5.5,2600 ISL BLVD WILLIAMS IS,4740.0,NaN,1997.0,NaN,654.0,6243.0,Sold,NaN,NaN,https://www.redfin.com/FL/A

## Time on Market Calculator

In [94]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 24) ## List (Earlier) date
date2 = datetime(2025, 6, 5) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

193


## Clean CSV for Datawrapper Chart

In [102]:
chart_df = df_top_ten

In [103]:
chart_df = chart_df.fillna(" ")

In [104]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [105]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [106]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [107]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [108]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [113]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [126]:
chart_df['PPSF'].astype(float)

ValueError: could not convert string to float: '$2,614'

In [116]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [124]:
desired_order = [
    'SOLD DATE',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [125]:
chart_df

,SOLD DATE,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-06-06,7852 Fisher Island Dr #7852 in Miami Beach,"$12,000,000","4,590","$2,614",1990,Jill Eber Coldwell Banker Realty,Alexa Iacovelli Luxe Living Realty
1,2025-06-05,18201 Collins Ave #409 in Sunny Isles Beach,"$5,600,000","3,087","$1,814",2009,Daniel Tzinker The Agency Florida LLC,Daniel Tzinker The Agency Florida LLC
2,2025-06-02,1000 S Pointe Dr #501 in Miami Beach,"$5,270,000","3,365","$1,566",2002,Jill Hertzberg Coldwell Banker Realty,MAR NON-MEMBER NO MLS PARTICIPATION (MAR)
3,2025-06-06,100 S Pointe Dr #907 in Miami Beach,"$4,800,000","1,870","$2,567",2003,Sonia Toth Brown Harris Stevens,Taylor Richardson The Corcoran Group
4,2025-06-02,2900 NE 7th Ave #407 in Miami,"$3,500,000","4,146",$844,2017,"Diego Guillen DGS Real Estate, LLC Andreina Es...","Diego Guillen DGS Real Estate, LLC Andreina Es..."
5,2025-06-03,901 Brickell Key Blvd #3508 in Miami,"$3,340,000","2,860","$1,168",2005,"Joao Carvalho Jad Realty, LLC. Daniela Carvalh...",Emerson De Lima Cervera Real Estate Inc.
6,2025-06-02,19225 Fisher Island Dr #19225 in Fisher Island,"$3,300,000","1,875","$1,760",1989,"Lazaro Rivero Rivero Real Estate , Inc. Rose B...",Giovanna Ingletto Amaral RGI Realty
7,2025-06-06,2600 Island Blvd #2306 in Aventura,"$3,100,000","4,740",$654,1997,"Joel Matus Williams Island Realty, LLC","Caryn Botknecht Compass Florida, LLC"
8,2025-06-02,1000 Brickell Plz Ph 5003 in Miami,"$2,736,000","2,076","$1,318",2019,"Miltiadis Kastanis Compass Florida, LLC.",Jose Fernand Fernandez Keller Williams Realty ...
9,2025-06-06,6000 Island Blvd #2102 in Aventura,"$2,700,000","2,840",$951,2005,"Joel Matus Williams Island Realty, LLC",Jill Meyerson-Barzilay One Sotheby's Internati...


In [118]:
csv_date_string = today.strftime("%m_%d_%Y")

In [119]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [121]:
chart_df.columns

Index(['SOLD DATE', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'YEAR BUILT', 'PPSF',
       'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')